In [2]:
# imports
import os
import numpy as np
import scipy.stats as st
import scipy.signal as ss
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axisartist.parasite_axes import HostAxes, ParasiteAxes

In [3]:
# Test data
txt_Data = "sample.txt"
xv = np.linspace(0, 2*np.pi, 500)
yv = 5*np.sin(xv)
ye = np.ones(500)/100
z = np.matrix([xv, yv, ye]).T
np.savetxt(txt_Data, z, delimiter=',')

In [4]:
def loadDat(txt_Data):
    '''Read data from .dat file [time, volume]'''
    # Parameters
    mass_error = 0.01 # g
    p = 998.2336 # Density of water
    p_error = 0.0001 # Error in density
    
    # load data file
    dat = np.loadtxt(txt_Data)
    
    
    error = np.ones(len(dat)) # Generate array of error values
    data = np.zeros([len(dat), 3]) # Initialise output data array
    # Set up data array [t, v=m/p, v uncertainty]
    for i in range(len(dat)):
        error = (dat[i,1]/(p*10**3))*np.sqrt((mass_error/dat[i,1])**2+(p_error/p)**2)
        data[i,0] = dat[i,0]/1000
        data[i,1] = dat[i,1]/(p*10**3)
        data[i,2] = error
    return data

In [5]:
# defines model object
class models:
    """Containes model functions and params (with initialiser)"""
    def __init__(self):
        self.float_Chi2 = 0
    
    ##########Define models here###########
    def polynomial(self, arr_X, arr_Vals):
        '''Polynumial model input x and values'''
        float_Y = 0.0
        for i in range(self.int_N):
            float_Y += arr_Vals[i]*arr_X**i
        return float_Y
    
    def harmonic(self, arr_X, arr_Vals):
        '''a*sin(b*x)+d*cos(e*x) model input x and values'''
        s = arr_Vals[0]*np.sin(arr_Vals[1]*arr_X)
        c = arr_Vals[2]*np.cos(arr_Vals[3]*arr_X)
        return s + c
    #######################################    
    
    def initVals(self, int_N):
        '''Initialiser for the parameters to be optimised for the model'''
        self.arr_Vals = np.random.rand(int_N)
        self.float_UnVal = np.random.rand(int_N)
        
    def setMod(self, mod, params):
        '''Initialises the users chosem model type'''
        if mod == 0:
            # Polynomial model with params[0] number of parameters
            self.initVals(self, params[0])
            self.int_N = params[0] # Degree of polynomial
            self.f = self.polynomial
        if mod == 1:
            self.initVals(self, 4)
            self.f = self.harmonic

In [6]:
def Chi2(arr_Vals, model, data):
    '''Chi^2 calculation'''
    float_Top = data[:,1]-model.f(model, data[:,0], arr_Vals)
    float_Chi = np.sum(np.power((float_Top/data[:,2]),2))
    return float_Chi

def Chi2a1(float_Val, model, data, fixIndx):
    '''Chi^2 modification function allowing all but one variable 
    to be manipulated by the optimiser used for Step 1 of algorithm 
    from lab session 2'''
    # Fills in the arr_Valls with the fixed parameter and all the free ones
    arr_Vals = model.arr_Vals
    arr_Vals[fixIndx] = float_Val # Replaces the value in arr_vals with the fixed value
    Chi = Chi2(arr_Vals, model, data)
    # Is optimised when Chi^2 calculated here approaches minimised Chi^2 + 1
    return abs(Chi - model.float_Chi2 - 1)

def Chi3(arr_Val, model, data, freeIndx):
    '''Chi^2 modification function allowing only one variable 
    to be manipulated by the optimiser used for Step 2 of algorithm 
    from lab session 2'''
    arr_Vals = model.arr_Vals
    arr_Vals[freeIndx] = arr_Val # Replaces the value in arr_vals with the free values
    Chi = Chi2(arr_Vals, model, data)
    # Is optimised when Chi^2 is minimised
    return Chi

In [7]:
def UncIndv(model, data, fixIndx):
    '''Performs iterative chi^2 uncertainty calculation'''
    arr_Vals = np.copy(model.arr_Vals) # initialise value array
    # Free index = [1,2,...,n-1,n+1,...m] where fixIndex = n
    freeIndx = np.delete(np.arange(0, len(arr_Vals), 1), fixIndx)  
    for i in range(10):
        # Step 1
        # Optimisation function
        dict_Unc1 = opt.minimize(Chi2a1,
                                   arr_Vals[fixIndx],
                                   method='nelder-mead',
                                   args=(model, data, fixIndx))
        
        # gets output from the optimisation
        arr_Vals[fixIndx] = dict_Unc1.x
        #Step 2
        # Optimisation function
        dict_Unc2 = opt.minimize(Chi3,
                                   arr_Vals[freeIndx],
                                   method='nelder-mead',
                                   args=(model, data, freeIndx))
        
        # gets output from the optimisation
        arr_Vals[freeIndx] = dict_Unc1.x
    
    # Perform Step 1 again to end on odd number
    # Optimisation function
    dict_Unc1 = opt.minimize(Chi2a1,
                           arr_Vals[fixIndx],
                           method='nelder-mead',
                           args=(model, data, fixIndx))
    return dict_Unc1.x

In [8]:
def Uncertainty(model, data):
    '''Uncertainty is calculated (iteratively0 using the method from lab session 2'''
    uncert = np.copy(model.arr_Vals) # Prevents overriding the memory
    for i in range(len(uncert)):
        uncert[i] -= UncIndv(model, data, i)
    return abs(uncert)

In [9]:
def Resid(arr_Vals, model, data):
    '''Calculates the residuals given a model with model params and data'''
    resid = np.ones(len(data))
    resid = (data[:,1] - model.f(model, data[:,0], arr_Vals))/data[:,2]  
    return resid

In [10]:
def linResid(arr_Vals, data):
    resid = np.ones(len(data[0]))
    resid = (data[1] - [arr_Vals[0]]*len(data[0]) - arr_Vals[1]*data[0])
    return resid

In [11]:
def DW(residuals):
    '''Calculates the Durbin_Watson statistic given an array of residuals'''
    num = 0
    for i in range(1, len(residuals)):
        num += (residuals[i]-residuals[i-1])**2
    den = np.sum(residuals**2)
    return num/den

In [12]:
# Optimises a model v(t) for a given data file
# The second parameter output corresponds to the gradient
# dv/dt with the associated uncertainty
# Input the data file below. Ensure the path is local
#################################
data = loadDat("./Black/490.dat")
#################################

# Initialise model
obj_Model = models
obj_Model.setMod(obj_Model, 0, [2])

# Optimise model
dict_Result = opt.minimize(Chi2,
                           obj_Model.arr_Vals,
                           method='nelder-mead',
                           args=(obj_Model, data))

# Storing model data
obj_Model.arr_Vals = dict_Result.x
obj_Model.float_Chi2 = dict_Result.fun
obj_Model.float_UnVal = Uncertainty(obj_Model, data)

# Output results
print("Chi^2: ", obj_Model.float_Chi2)
print("DOF: ", len(data) - len(obj_Model.arr_Vals))
print("Reduced Chi^2: ", obj_Model.float_Chi2/(len(data) - len(obj_Model.arr_Vals)))
print("parameters:")
for i in range(len(obj_Model.arr_Vals)):
    print("    ", obj_Model.arr_Vals[i], "+-", obj_Model.float_UnVal[i])

OSError: ./Black/490.dat not found.

In [ ]:
# Plots valume against time for the model and the data
# Generates t values for model
x = np.linspace(np.amin(data[:,0]), np.amax(data[:,0]), 100)
plt.plot(data[:,0], data[:,1], color='b', label='Data') # Data
plt.scatter(data[:,0], data[:,1], color='b')
plt.plot(x, obj_Model.f(obj_Model, x, obj_Model.arr_Vals), color='r', label='Model') # Model
plt.xlabel('time $(s)$')
plt.ylabel('volume $(m^3)$')
plt.title("Expelled volume of water against time")
plt.legend()
plt.show()

In [ ]:
# Plots the normalised residuals for the desired sample
res = Resid(obj_Model.arr_Vals, obj_Model, data)
plt.xlabel("time $(s)$")
plt.ylabel("Normalised residual")
plt.title("Normalised residuals")
plt.plot(data[:,0], res)
plt.scatter(data[:,0], res)
plt.show()

In [ ]:
peaks = ss.find_peaks(res)
t = []
for i in range(len(peaks[0])-1):
    t.append(peaks[0][i+1]-peaks[0][i])
avrT = np.average(t)
print(avrT)

In [ ]:
# Calculates viscosity using two methods; from the gradient of (h, dv/dt) graph
# and from average value of 1/h*dv/dt (CALC1 and CALC2 respectively).
# Record data as h.dat where h is in 10^(-4)m
# Input the directory corresponding the the desired data set
# Ensure that the selected directory is in the same folder as this script
######################
directory = 'Red1'
######################

# Parameters used to calculate viscosity
tubeParams = {'Black': [15.2*10**(-2), 1.03*10**(-3), 0.04*10**(-3)],
             'Red1': [14.1*10**(-2), 0.471*10**-3, 0.006*10**(-3)],
             'Blue': [15.2*10**(-2), 0.814*10**(-3), 0.01*10**(-3)]}

hUnc = 0.075*10**(-2) # Uncertainty in height measurements
a = tubeParams[directory][1:3]
p = [998.2336, 0.0001]
g = [9.81, 0.1]
l = [tubeParams[directory][0], 0.71*10**(-2)]
pi = [np.pi, 0]
var = np.array([a,p,g,l,pi])


# Accepted viscosity
accVisc = 1.002*10**(-3)
accViscUnc = 0.0001*10**(-3)

# Initialise model for finding volume v(t)
obj_Model_1 = models
obj_Model_1.setMod(obj_Model_1, 0, [2])

# Array initialisation for for loop
gradH = []
data_hdvdt = []

for file in os.listdir(directory):
    # Load data
    data = loadDat(os.path.join(directory, file))
    
    obj_Model_1.initVals(obj_Model_1, 2)
    
    # Optimise fit
    dict_Result = opt.minimize(Chi2,
                           obj_Model_1.arr_Vals,
                           method='nelder-mead',
                           args=(obj_Model_1, data)) # Optimisation function
    
    # Degrees of freedom calculation
    DOF = len(data) - len(obj_Model_1.arr_Vals)
    
    # Assign optimised values to model object variables
    obj_Model_1.float_Chi2 = dict_Result.fun
    obj_Model_1.arr_Vals = dict_Result.x
    
    # Determines h from file name
    h = float(file[:len(file)-3])/10000 #- 1.6*10**(-2)
    
    grad = dict_Result.x[1]
    gradUn = Uncertainty(obj_Model_1, data)[1] # Gradient uncertainty calculation
    gradHUn = (grad/h)*((gradUn/grad)**2 + (hUnc/h)**2)**0.5 # Gradient/h
    
    # Appends values required later to arrays
    gradH.append([grad/h, gradHUn]) # Contains 1/h*dv/dt and uncertainty used in CALC2
    data_hdvdt.append([h, grad, gradUn]) # Contains height, dv/dt and uncertainty used in CALC1
    
    # Output and plot
    # Used to plot volume against time see below also
    '''
    plt.plot(data[:,0], obj_Model_1.f(obj_Model_1, data[:,0], obj_Model_1.arr_Vals))
    plt.errorbar(data[:,0], data[:,1], yerr=data[:,2], ms=50)
    '''
    print('Height:', np.around(h, decimals=3), ' DOF:', DOF, ' Reduced Chi^2:', dict_Result.fun/DOF)

# Clear new line in output
print("")
    
# Convert arrays to np arrays for slicing
gradH = np.array(gradH)
data_hdvdt1 = np.array(data_hdvdt)


                    ###CALC1###
    
# Calculate viscosity from gradient of (h, dv/dt)
# Begin by performing chi^2 analysis on the (h, dv/dt) plot
obj_Model_hdvdt1 = models
obj_Model_hdvdt1.setMod(obj_Model_hdvdt1, 0, [2])
dict_Result_hdvdt1 = opt.minimize(Chi2,
                           obj_Model_hdvdt1.arr_Vals,
                           method='nelder-mead',
                           args=(obj_Model_hdvdt1, data_hdvdt1)) # Optimisation function

# Getting values from the minimisation class
obj_Model_hdvdt1.float_Chi2 = dict_Result_hdvdt1.fun
obj_Model_hdvdt1.arr_Vals = dict_Result_hdvdt1.x

grad_hdvdt1 = dict_Result_hdvdt1.x[1] # Gradient as calculated with chi^2
gradUn_hdvdt1 = Uncertainty(obj_Model_1, data_hdvdt1)[1] # Uncertainty as calculated by Chi^2

# Calcualte the viscosity
viscosity_hdvdt1 = (pi[0]*p[0]*g[0]*a[0]**4)/(grad_hdvdt1*8*l[0])

# Uncertainty calculation
temp = np.sum((var[:,1]/var[:,0])**2) + 3*(var[0,1]/var[0,0])**2 + (gradUn_hdvdt1/grad_hdvdt1)**2
viscosity_hdvdt1_unc = viscosity_hdvdt1*np.sqrt(temp)

# Goodness of fit
compError = (viscosity_hdvdt1-accVisc)/viscosity_hdvdt1_unc

# Output
red1Vals = obj_Model_hdvdt1.arr_Vals
print("Reduced Chi^2 for fitting (h, dv/dt): ", dict_Result_hdvdt1.fun/6)
print("viscosity from gradient: ", viscosity_hdvdt1, "+-", viscosity_hdvdt1_unc, "Pa")
print('Goodness of fit:', compError, 'sigma', '\n')

                    ###CALC1###

In [ ]:
# Calculates viscosity using two methods; from the gradient of (h, dv/dt) graph
# and from average value of 1/h*dv/dt (CALC1 and CALC2 respectively).
# Record data as h.dat where h is in 10^(-4)m
# Input the directory corresponding the the desired data set
# Ensure that the selected directory is in the same folder as this script
######################
directory = 'Black'
######################

# Parameters used to calculate viscosity
tubeParams = {'Black': [15.2*10**(-2), 1.03*10**(-3), 0.04*10**(-3), 0, 'k'],
             'Red': [14.1*10**(-2), 0.471*10**-3, 0.006*10**(-3), 1, 'r'],
             'Blue': [15.2*10**(-2), 0.814*10**(-3), 0.01*10**(-3), 2, 'b']}

# Initialise array of output values
viscAndUnc = np.empty([len(tubeParams), 2])
res = np.empty([len(tubeParams), 10, 50])
avrT = np.empty([len(tubeParams), 10])
stdT = np.empty([len(tubeParams), 10])
sumCompError = 0

# Initialise figure
g1 = 1 # (h, dv/dt)
g2 = 2 # (h, residuals)
g3 = 0 # (h, drop volume)
fig = plt.figure(figsize=[8,10])
gs = gridspec.GridSpec(3, 1)
gs.update(wspace=0.0, hspace=0.0)
axes = [fig.add_subplot, fig.add_subplot, fig.add_subplot]

axes[1] = fig.add_subplot(gs[1])
axesSub = plt.axes([0.185, 0.52, 0.2, 0.1])
axesSub.set_ylim(bottom=0.55, top=0.7)
axesSub.set_xlim(left=0.06, right=0.08)
axes[0] = fig.add_subplot(gs[0], sharex=axes[1])
axes[2] = fig.add_subplot(gs[2], sharex=axes[1])
plt.xlabel('Height / $m$')
axes[g1].set_ylabel('$dv/dt$ / $mm^3 s^{-1}$')
axes[g2].set_ylabel('log transformed normalised residuals')
axes[g3].set_ylabel('Drip volume / $mm^{3}$')
plt.setp(axes[g1].get_xticklabels(), visible=False)
plt.setp(axes[g3].get_xticklabels(), visible=False)
axes[g3].set_ylim(-5, 45)
axes[g3].spines['right'].set_visible(False)
axes[g3].spines['top'].set_visible(False)
axes[g1].spines['right'].set_visible(False)
axes[g2].spines['right'].set_visible(False)

for countDir, directory in enumerate(tubeParams):
    print('Tube:', directory)
    hUnc = 0.075*10**(-2) # Uncertainty in height measurements
    a = tubeParams[directory][1:3]
    p = [998.2336, 0.0001]
    g = [9.81, 0.1]
    l = [tubeParams[directory][0], 0.71*10**(-2)]
    pi = [np.pi, 0]
    var = np.array([a,p,g,l,pi])

    # Accepted viscosity
    accVisc = 1.002*10**(-3)
    accViscUnc = 0.0001*10**(-3)

    # Initialise model for finding volume v(t)
    obj_Model_1 = models
    obj_Model_1.setMod(obj_Model_1, 0, [2])

    # Array initialisation for for loop
    gradH = []
    data_hdvdt = []
    chi_10 = np.array([])

    for countFile, file in enumerate(os.listdir(directory)):
        # Load data
        data = loadDat(os.path.join(directory, file))

        obj_Model_1.initVals(obj_Model_1, 2)

        # Optimise fit
        dict_Result = opt.minimize(Chi2,
                               obj_Model_1.arr_Vals,
                               method='nelder-mead',
                               args=(obj_Model_1, data)) # Optimisation function

        # Degrees of freedom calculation
        DOF = len(data) - len(obj_Model_1.arr_Vals)

        # Assign optimised values to model object variables
        obj_Model_1.float_Chi2 = dict_Result.fun
        obj_Model_1.arr_Vals = dict_Result.x
        
        # Fill chi_10 with each of the 10 chi^2 values for each file
        chi_10 = np.append(chi_10, obj_Model_1.float_Chi2)

        # Residual analysis
        res[countDir, countFile] = Resid(obj_Model_1.arr_Vals, obj_Model_1, data)
        peaks = ss.find_peaks(res[countDir, countFile])
        t = []
        for i in range(len(peaks[0])-1):
            t.append(peaks[0][i+1]-peaks[0][i])
        avrT[countDir, countFile] = np.average(t)
        stdT[countDir, countFile] = np.std(t)

        
        # Determines h from file name
        h = float(file[:len(file)-3])/10000 #- 1.6*10**(-2)

        grad = dict_Result.x[1]
        gradUn = Uncertainty(obj_Model_1, data)[1] # Gradient uncertainty calculation
        gradHUn = (grad/h)*((gradUn/grad)**2 + (hUnc/h)**2)**0.5 # Gradient/h

        # Appends values required later to arrays
        gradH.append([grad/h, gradHUn]) # Contains 1/h*dv/dt and uncertainty used in CALC2
        data_hdvdt.append([h, grad, gradUn]) # Contains height, dv/dt and uncertainty used in CALC1

        # Output and plot
        # Used to plot volume against time see below also
        '''
        plt.plot(data[:,0], obj_Model_1.f(obj_Model_1, data[:,0], obj_Model_1.arr_Vals))
        plt.errorbar(data[:,0], data[:,1], yerr=data[:,2], ms=50)
        '''
        
        # Output data for each file
        print('Height:', np.around(h, decimals=3), ' DOF:', DOF, ' Reduced Chi^2:', dict_Result.fun/DOF)
        sumCompError += dict_Result.fun/DOF

    # Clear new line in output
    print("")

    # Convert arrays to np arrays for slicing
    gradH = np.array(gradH)
    data_hdvdt = np.array(data_hdvdt)


                        ###CALC1###

    # Calculate viscosity from gradient of (h, dv/dt)
    # Begin by performing chi^2 analysis on the (h, dv/dt) plot
    obj_Model_hdvdt = models
    obj_Model_hdvdt.setMod(obj_Model_hdvdt, 0, [2])
    dict_Result_hdvdt = opt.minimize(Chi2,
                               obj_Model_hdvdt.arr_Vals,
                               method='nelder-mead',
                               args=(obj_Model_hdvdt, data_hdvdt)) # Optimisation function

    # Getting values from the minimisation class
    obj_Model_hdvdt.float_Chi2 = dict_Result_hdvdt.fun
    obj_Model_hdvdt.arr_Vals = dict_Result_hdvdt.x

    grad_hdvdt = dict_Result_hdvdt.x[1] # Gradient as calculated with chi^2
    gradUn_hdvdt = Uncertainty(obj_Model_1, data_hdvdt)[1] # Uncertainty as calculated by Chi^2

    # Calcualte the viscosity
    viscosity_hdvdt = (pi[0]*p[0]*g[0]*a[0]**4)/(grad_hdvdt*8*l[0])

    # Uncertainty calculation
    temp = np.sum((var[:,1]/var[:,0])**2) + 3*(var[0,1]/var[0,0])**2 + (gradUn_hdvdt/grad_hdvdt)**2
    viscosity_hdvdt_unc = viscosity_hdvdt*np.sqrt(temp)

    # Goodness of fit
    compError = (viscosity_hdvdt-accVisc)/viscosity_hdvdt_unc
    chi_hdvdt = dict_Result_hdvdt.fun
    
    # Append results to array
    viscAndUnc[tubeParams[directory][3]] = [viscosity_hdvdt, viscosity_hdvdt_unc]
    
    # Output
    print("Reduced Chi^2 for fitting (h, dv/dt): ", chi_hdvdt/8)
    print("viscosity from gradient: ", viscosity_hdvdt, "+-", viscosity_hdvdt_unc, "Pa")
    print('Goodness of fit:', compError, 'sigma', '\n')

                        ###CALC1###


                        ###CALC2###

    # Calculate viscosity from average 1/h*dv/dt
    # Calculate average of (1/h)*dV/dt with uncertainty
    avGradH = np.average(gradH[:,0]) # gradH[:,0] are 1/h*dv/dt values
    avGradHUn = (np.sqrt(np.sum(gradH[:1]**2))/np.sum(gradH))*avGradH # gradH[:,1] are uncertainties in 1/h*dv/dt values

    # Viscosity calculation
    viscosity = (pi[0]*p[0]*g[0]*a[0]**4)/(8*l[0]*avGradH)

    # Uncertainty calculation
    temp = np.sum((var[:,1]/var[:,0])**2) + 3*(var[0,1]/var[0,0])**2 + (avGradHUn/avGradH)**2
    viscosityUn = viscosity*np.sqrt(temp)

    # Goodness of fit
    compError = (viscosity-accVisc)/viscosityUn
    
    # Output
    print('Viscosity from average:', viscosity, '+-', viscosityUn, 'Pa')
    print('Goodness of fit:', compError, 'sigma')

                        ###CALC2###


    # Used to view the plots of volume against time see above also
    '''
    plt.xlabel("Time / $s$")
    plt.ylabel("Water Voulme / $m^3$")
    '''
    
    # PLOTTING
    # Plot dv/dt against h
    hdvdtModel = 10**7*obj_Model_hdvdt.f(obj_Model_hdvdt,
                                                data_hdvdt[:,0],
                                                obj_Model_hdvdt.arr_Vals)

    #axes[g1].plot(data_hdvdt[:,0], 10**4*data_hdvdt[:,1], color=tubeParams[directory][4], label='Data') # Data
    axes[g1].errorbar(data_hdvdt[:,0], 10**7*data_hdvdt[:,1], yerr=data_hdvdt[:,2],
                      color=tubeParams[directory][4], label='Data', fmt='.')
    axes[g1].plot(data_hdvdt[:,0], hdvdtModel, color='grey', label='Model', alpha=0.75, linestyle='--') # Model
    if directory == 'Red':
        l = len(data_hdvdt)
        axesSub.plot(data_hdvdt[l-5:l,0], 10**7*data_hdvdt[l-5:l,1], color=tubeParams[directory][4])

    # Plot drop period
    area = chi_10/20
    # Error is taken to be the percentage error in the avrT
    errorV = np.sqrt((stdT[countDir]/avrT[countDir])**2)*avrT[countDir]*hdvdtModel
    #axes[g3].plot(data_hdvdt[:,0], avrT[countDir]*hdvdtModel, color=tubeParams[directory][4])
    axes[g3].scatter(data_hdvdt[:,0], avrT[countDir]*hdvdtModel,
                    s=area,
                    alpha=0.5,
                    color=tubeParams[directory][4])
    axes[g3].errorbar(data_hdvdt[:,0], avrT[countDir]*hdvdtModel, yerr=errorV, color=tubeParams[directory][4], fmt='.')
    
    # Get DW for the chi^2 against height and drip volume
    arrDripsChi = st.linregress(avrT[countDir]*hdvdtModel, area)
    linRes = linResid([arrDripsChi[1], arrDripsChi[0]], [avrT[countDir]*hdvdtModel, area])
    print('DW drip volume:', DW(linRes))
    
    # Plot normalised residuals
    if directory == 'Red':
        res_hdvdt = Resid(red1Vals, obj_Model_hdvdt1, data_hdvdt1)
        axes[g2].scatter(data_hdvdt1[:,0], (np.sign(res_hdvdt)*np.log(abs(res_hdvdt)+1)),
                     color=tubeParams[directory][4], s=30, marker='x')
        print('DW Red1 dv/dt against h:', DW(res_hdvdt))
    res_hdvdt = Resid(obj_Model_hdvdt.arr_Vals, obj_Model_hdvdt, data_hdvdt)
    axes[g2].scatter(data_hdvdt[:,0], (np.sign(res_hdvdt)*np.log(abs(res_hdvdt)+1)),
                     color=tubeParams[directory][4], s=30, marker='.')

    print('DW dv/dt against h:', DW(res_hdvdt))
    print('Average \chi^2:', sumCompError/10)
    sumCompError=0
    print('\n')
    
    #axes[g2].hist(res_hdvdt, bins=4, density=True,
    #              color=tubeParams[directory][4],
    #             alpha=0.25,
    #             orientation='horizontal')

#Draw reference line on residual plot
axes[g2].plot([0.02, 0.1], [0, 0], linestyle='--', color='grey', alpha=0.75)
one = np.array([1,1])
axes[g2].plot([0.02, 0.1], (np.sign(one)*np.log(abs(one)+1)),
              linestyle='--', color='grey', alpha=0.75)
axes[g2].plot([0.02, 0.1], (np.sign(-one)*np.log(abs(-one)+1)),
              linestyle='--', color='grey', alpha=0.75)

# Calculates average viscosity
avVisc = np.average(viscAndUnc[:,0])
# Calculates uncertainty in average viscosity
uncAvVisc = avVisc*(np.sqrt(np.sum(viscAndUnc[:,1]**2))/np.sum(viscAndUnc[:,0]))

# Calculates compartive error in number of standard deviations
compError = (avVisc-accVisc)/uncAvVisc

# Output results
print('Average')
print('Average Viscosity:', avVisc, '+-', uncAvVisc, 'Pa')
print('Goodness of fit:', compError, 'sigma')

plt.xlim([0.023,0.09])
plt.plot()
plt.show();

In [ ]:
# Calculates average viscosity
# Populate array with viscosity and uncertainty for each tube
#viscAndUnc = np.array([[0.0010854513730798512, 6.496738072504902e-05],
#                      [0.0035071062467488512, 0.00019659333640224117],
#                      [0.0021392328905156317, 0.00019805147418198288]])

# Calculates average viscosity
avVisc = np.average(viscAndUnc[:,0])
# Calculates uncertainty in average viscosity
uncAvVisc = avVisc*(np.sqrt(np.sum(viscAndUnc[:,1]**2))/np.sum(viscAndUnc[:,0]))

# Calculates compartive error in number of standard deviations
compError = (avVisc-accVisc)/uncAvVisc

# Output results
print('Viscosity:', avVisc, '+-', uncAvVisc, 'Pa')
print('Goodness of fit:', compError, 'sigma')
